# Import PySpark

In [128]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Initialize PySpark Session

In [7]:
spark = SparkSession.builder.appName("zillow").getOrCreate()

# Load Data

In [14]:
zillow = (spark
    .read
    .option("header", True)
    .csv(r"G:\My Drive\AUEB\3. AUEB MSCDS\Studying Material\Courses\Winter\Large Scale Data Management\Assignments\A2_pyspark\zillow.csv"))

# Create User Defined Functions

In [ ]:
@udf(returnType = FloatType()) 
def GrossInternalArea(input):
    import re
    try:
        return float(re.search("([0-9]*)\ssqft", input).group(1))
    except:
        return -1

In [ ]:
zillow.select(GrossInternalArea("facts and features")).show(n = 10)

+-------------------------------------+
|GrossInternalArea(facts and features)|
+-------------------------------------+
|                                705.0|
|                               1228.0|
|                               1000.0|
|                               6836.0|
|                               1000.0|
|                               2313.0|
|                                780.0|
|                                856.0|
|                                675.0|
|                                511.0|
+-------------------------------------+
only showing top 10 rows



## Extract Number of Bedrooms

In [71]:
@udf(returnType = FloatType()) 
def NumberOfBedrooms(input):
    import re
    try:
        return float(re.search("([0-9]*)\sbds", input).group(1))
    except:
        return -1

In [72]:
zillow.select(NumberOfBedrooms("facts and features")).show(n = 10)

+------------------------------------+
|NumberOfBedrooms(facts and features)|
+------------------------------------+
|                                 2.0|
|                                 2.0|
|                                 1.0|
|                                 4.0|
|                                 2.0|
|                                 3.0|
|                                 2.0|
|                                 2.0|
|                                 1.0|
|                                 2.0|
+------------------------------------+
only showing top 10 rows



## Extract Number of Bathrooms

In [69]:
@udf(returnType = FloatType()) 
def NumberOfBathrooms(input):
    import re
    try:
        return float(re.search("([0-9]*\.[0-9]*)\sba", input).group(1))
    except:
        return -1

In [70]:
zillow.select(NumberOfBathrooms("facts and features")).show(n = 10)

+-------------------------------------+
|NumberOfBathrooms(facts and features)|
+-------------------------------------+
|                                  1.0|
|                                  2.0|
|                                  1.0|
|                                  7.0|
|                                  3.0|
|                                  3.0|
|                                  1.0|
|                                  1.0|
|                                  1.0|
|                                  1.0|
+-------------------------------------+
only showing top 10 rows



## Extract Gross Internal Area

In [73]:
@udf(returnType = FloatType()) 
def GrossInternalArea(input):
    import re
    try:
        return float(re.search("([0-9]*)\ssqft", input).group(1))
    except:
        return -1

In [74]:
zillow.select(GrossInternalArea("facts and features")).show(n = 10)

+-------------------------------------+
|GrossInternalArea(facts and features)|
+-------------------------------------+
|                                705.0|
|                               1228.0|
|                               1000.0|
|                               6836.0|
|                               1000.0|
|                               2313.0|
|                                780.0|
|                                856.0|
|                                675.0|
|                                511.0|
+-------------------------------------+
only showing top 10 rows



## Extract Type

In [79]:
@udf(returnType = StringType()) 
def Type(input):
    import re
    try:
        return re.search("(.*)\sfor sale", input).group(1)
    except:
        return input

In [80]:
zillow.select(Type("title")).show(n = 10)

+-----------+
|Type(title)|
+-----------+
|      Condo|
|      Condo|
|      Condo|
|      House|
|      Condo|
|      House|
|      Condo|
|      Condo|
|      Condo|
|      Condo|
+-----------+
only showing top 10 rows



## Extract Offer

In [81]:
@udf(returnType = StringType()) 
def Offer(input):
    import re
    try:
        if bool(re.search("sale", input)):
            return "sale"
        elif bool(re.search("rent", input)):
            return "rent"
        elif bool(re.search("sold", input)):
            return "sold"
        else:
            return "foreclose"
    except:
        return input

In [82]:
zillow.select(Offer("title")).show(n = 10)

+------------+
|Offer(title)|
+------------+
|        sale|
|        sale|
|        sale|
|        sale|
|        sale|
|        sale|
|        sale|
|        sale|
|        sale|
|        sale|
+------------+
only showing top 10 rows



## Extract Price

In [83]:
@udf(returnType = IntegerType()) 
def Price(input):
    import re
    return int(re.search("([0-9,]+[0-9]+)", input).group(1).replace(",", ""))

In [84]:
zillow.select(Price("price")).show(n = 10)

+------------+
|Price(price)|
+------------+
|      342000|
|     1700000|
|      336500|
|     9950000|
|      479000|
|      899000|
|      397300|
|      619900|
|      850000|
|      649900|
+------------+
only showing top 10 rows



## Filter Out Listings That Are Not For Sale

In [85]:
zillow.select("*").where(Offer("title") == "sale").show(n = 10)

+--------------+-------+------------+-----+-----------+-----------+--------------------+--------------------+--------------------+
|         title|address|        city|state|postal_code|      price|  facts and features|real estate provider|                 url|
+--------------+-------+------------+-----+-----------+-----------+--------------------+--------------------+--------------------+
|Condo for sale|   null|  Somerville|   MA|       2145|  $342,000 |2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2116|$1,700,000 |2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2118|  $336,500 |1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|
|House for sale|   null|      Boston|   MA|       2118|$9,950,000 |4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2128|  $479,000 |2 bds, 3.0 ba ,1

## Filter Out Listings With More Than 10 Bedrooms

In [87]:
zillow.select("*").where(NumberOfBedrooms("facts and features") <= 10).show(n = 10)

+--------------+-------+------------+-----+-----------+-----------+--------------------+--------------------+--------------------+
|         title|address|        city|state|postal_code|      price|  facts and features|real estate provider|                 url|
+--------------+-------+------------+-----+-----------+-----------+--------------------+--------------------+--------------------+
|Condo for sale|   null|  Somerville|   MA|       2145|  $342,000 |2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2116|$1,700,000 |2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2118|  $336,500 |1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|
|House for sale|   null|      Boston|   MA|       2118|$9,950,000 |4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2128|  $479,000 |2 bds, 3.0 ba ,1

## Filter Out Listings With Price Greater Than 20000000 And Lower Than 100000

In [89]:
zillow.select("*").where((Price("price") >= 100000) & (Price("price") <= 20000000)).show(n = 10)

+--------------+-------+------------+-----+-----------+-----------+--------------------+--------------------+--------------------+
|         title|address|        city|state|postal_code|      price|  facts and features|real estate provider|                 url|
+--------------+-------+------------+-----+-----------+-----------+--------------------+--------------------+--------------------+
|Condo for sale|   null|  Somerville|   MA|       2145|  $342,000 |2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2116|$1,700,000 |2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2118|  $336,500 |1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|
|House for sale|   null|      Boston|   MA|       2118|$9,950,000 |4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|       2128|  $479,000 |2 bds, 3.0 ba ,1

## Calculate Average Price Per Sqft For Houses For Sale Grouping Them By The Number Of Bedrooms

In [139]:
(zillow
 .select(Price("price").alias("p"),
         GrossInternalArea("facts and features").alias("gia"),
         NumberOfBedrooms("facts and features").alias("nbds"))
 .where((Type("title") == "House") & (Offer("title") == "sale"))) \
 .groupBy("nbds") \
 .agg((avg("p") / avg("gia")).alias("avgpsqft")) \
 .orderBy("nbds") \
 .show()

+----+------------------+
|nbds|          avgpsqft|
+----+------------------+
| 0.0|            1250.0|
| 2.0| 713.7979122566344|
| 3.0| 697.9300109064761|
| 4.0|1125.3151402657086|
| 5.0|1286.3502045373516|
| 6.0| 529.4372004156471|
| 7.0|1126.0252348993288|
| 8.0|1567.6470588235295|
| 9.0|1108.1412183984853|
|11.0| 433.6545589325426|
+----+------------------+

